# COMPSCI 361 A4 S1 2021
jrow952
Jacob Rowland
364505466

## Task 1
Find the frequent itemsets from the given dataset. Minsup = 0.15 and minconf = 0.80

Working is included in Appendix 1 and 2 of the pdf.


## Task 2

Implement the Apriori algorithm

In [2]:
import csv
import itertools
import time
import pandas as pd

In [4]:
class AssociationRule():
    """
    Represents an association rule in the form {body} -> {head}

    Parameters:
    body (set): The set of items in the body of the association rule
    head (set): The set of items in the head of the association rule

    """
    def __init__(self, body:set, head:set):
        self.head = head
        self.body = body
        self.itemset = body.union(head)
        self.confidence = None
        self.support = None
        self.lift = None

    def __str__(self) -> str:
        return "{} -> {}".format(self.body, self.head)

This AssociaionRule class is used for both the Task 2 Apriori and ExtendedApriori in task 5.

In [5]:
class Apriori():
    """
    This object implements the Apriori unsupervised machine learning algorithm 
    for frequent item set mining and association rule learning
    """
    def __init__(self, minsup:float, minconf:float, minlift:float, path:str):
        """Initalise the Apiori class

        Parameters:
        minsup (float): Minimuim support
        minconf (float): Minimum confidence
        minlift (float): Minimum lift
        path (str): Transaction CSV path - each line is transaction of items seperated by comma
        items (set): The set of unique items found across all transactions - initalised by generateUniqueItemSet()
        transactions (list): A list of sets for each transaction - where each is a set of items

        Returns:
        None:Returning value
        """
        self.minsup = minsup
        self.minconf = minconf
        self.minlift = minlift
        self.path = path
        self.transactions = []
        self.items = []
        self.importTransactions()
        self.generateUniqueItemSet()

    def run(self):
        """
        Runs the Apriori itemset frequency algorithm and returns a sorted list of association rules that pass the minsup, minconf and minlift rules
        """
        print("Running Apriori...")
        print(" a. Finding association rules from {} transactions and {} unique items...".format(len(self.transactions), len(self.items)))
        frequentSets = self.generateFrequentSets() # find all frequent itemsets of length 1 to k-1
        print(" b. Generating association rules from frequent itemsets...")
        associationRules = self.generateAssociationRules(frequentSets) # generate rules that pass the minsup, minlift and minsup thresholds
        print(" c. Sorting association rules...")
        associationRules = self.sortAssociationRules(associationRules) # sorts rules by length, lift, confidence and support
        print("Complete.")
        return frequentSets, associationRules

    def generateAssociationRules(self, frequentSets:list)->list:
        """
        Generates a list of association rules that meet the confidence and lift thresholds

        Parameters:
        frequentSets (list): A list of frequent itemsets

        Returns:
        list: A list containing the association rules that pass the support, confidence and lift thresholds
        """
        associationRules = []
        # partition the set  
        for itemset in frequentSets:
            for i in range(len(itemset)):
                for c in [c for c in itertools.combinations(itemset, i+1)]:
                    body = set(list(c))
                    head = set([i for i in itemset if not i in c])
                    if (len(head) > 0 and len(body) > 0):
                        # from head/body partitions create association rules 
                        associationRule = AssociationRule(body, head) # X -> Y
                        associationRule.support = self.calculateSupport(associationRule.itemset)
                        associationRule.confidence = self.calculateConfidence(associationRule.itemset, associationRule.body)
                        associationRule.lift = self.calculateLift(associationRule.body, associationRule.head, associationRule.confidence)
                        # test is rule passes minconf, minsup and minlift
                        if (associationRule.confidence >= self.minconf) and (associationRule.support >= self.minsup) and (associationRule.lift >= self.minlift):
                            associationRules.append(associationRule)
        return associationRules

    def generateFrequentSets(self)->list:
        """
        Implements the Apriori frequent itemset generation algorithm.
        Where itemsets is c_k and frequentSets = l_k 

        Returns:
        list: A list of frequent itemsets of size k-1
        """
        allFrequentSets = [] 

        # Generate frequent itemsets of length k
        frequentSets, infrequentSets = self.eliminateCandidates(self.items) # c_1
        for fset in frequentSets:
            allFrequentSets.append(fset)
        # Repeat until no new frequent itemsets are identified
        k = 1
        while True:  
            k += 1
            print("     k = " + str(k) + "    " , end="\n")
            # Generate length (k+1) candidate itemsets from length k frequent itemsets
            itemsets = self.generateItemSets(frequentSets, k)
            # Prune candidate itemsets containing subsets of length k that are infrequent
            itemsets = self.prune(itemsets, infrequentSets)
            # Count the support of each candidate by scanning the DB
            # Eliminate candidates that are infrequent, leaving only those that are frequent
            prevFrequentSets = frequentSets.copy() # holds a copy of frequent sets for k-1
            frequentSets, infrequentSets = self.eliminateCandidates(itemsets)
            # Runs until no frequent itemsets are identified
            if len(frequentSets) == 0:
                frequentSets = prevFrequentSets
                break
            # stores all the frequent itemsets of size k
            for fset in frequentSets:
                allFrequentSets.append(fset)
        return allFrequentSets

    def sortAssociationRules(self, associationRules:list)->list: # TODO: sorted output list
        """
        Sorts the given list of association rules by four different attributes.
        Including by itemset length (decreasing), minlift, minconf and minsup (decreasing)

        Paramaters:
        associationRules (list): A list of AssociationRule objects

        Returns:
        list: A sorted list of AssociationRule objects

        """
        associationRules = sorted(associationRules, key=lambda x: x.support, reverse=True) # support (decreasing)
        associationRules = sorted(associationRules, key=lambda x: x.confidence, reverse=False) # confidence
        associationRules = sorted(associationRules, key=lambda x: x.lift, reverse=False) # lift value
        associationRules = sorted(associationRules, key=lambda x: len(x.itemset), reverse=True) # number of items (decreasing)

        return associationRules
    
    def eliminateCandidates(self, itemsets:list)->tuple:
        """
        Sorts candiadate itemsets by calculating the support value and comparing to the mininimum support value

        Parameters:
        itemsets (list): A list of sets to sort

        Returns:
        tuple: Returns a tuple of two sets. Containing the eliminated rules and one containing rules that pass
        """
        frequentSets = []
        infrequentSets = []
        for i in range(len(itemsets)):
            if not self.calculateSupport(itemsets[i]) < self.minsup:
                frequentSets.append(itemsets[i])
            else:
                infrequentSets.append(itemsets[i])
        return frequentSets, infrequentSets

    def generateItemSets(self, items:set, k:int)->list:
        """
        Generates combinations of itemsets and returns a list of sets

        Parameters
        items (set): The set of items to generate combinations from
        k (int): The length of the generated sets

        Returns:
        list: List of the generated combinations of length k
        """
        pairs = list(itertools.combinations(items, 2))

        itemsets = []
        for pair in pairs:
            set1 = pair[0]
            set2 = pair[1]
            set3 = set1.union(set2)
            if set3 not in itemsets and len(set3) == k:
                itemsets.append(set3)
        return itemsets

    def generateUniqueItemSet(self):
        """
        Generates a set of items that appear across all transactions. Updates the class attribute self.transactions
        """
        items = set()
        for transaction in self.transactions:
            for item in transaction:
                if item not in self.items:
                    items.add(item)
        for item in items:
            s = set()
            s.add(item)
            self.items.append(s)

    def importTransactions(self):
        """
        Reads in the csv file of transactions. The file is assumed to have no header. Each row is a set of items contained within a transaction
        """
        with open(self.path, 'r', newline='') as f:
            reader = csv.reader(f)
            for row in reader:
                cleanedRow = []
                for item in row:
                    cleanedRow.append(item.strip().upper())
                self.transactions.append(set(cleanedRow))

    def calculateSupport(self, itemset:set)->float:
        """
        Calculates the support value of a set by dividing the number of transactions a set occurs in with the total number of transactions

        Parameters
        itemset (set): A set of items

        Returns:
        float: The support value for that set (rule)

        """
        return self.count(itemset) / len(self.transactions)

    def calculateConfidence(self, itemset:set, body:set)->float:
        """
        Calculates how often items in Y appear in transactions containing X

        Parameters
        set (set): A set of items

        Returns
        float: The confidence value for that set (rule)

        """
        return self.calculateSupport(itemset) / self.calculateSupport(body)

    def calculateLift(self, body:list, head:list, confidence:float)->float:
        """
        Calculates the importance of a rule
        """
        bodySupport = self.calculateSupport(body)
        headSupport = self.calculateSupport(head)
        return confidence / ((bodySupport * headSupport) / bodySupport)

    def count(self, s:set)->int:
        """
        Parameters
        set (set): A set of items

        Returns:
        int: Frequency count for how many times s is a subset of a transaction t
        """
        count = 0
        for transaction in self.transactions:
            if set(s).issubset(transaction):
                count += 1
        return count

    def prune(self, itemsets:list, infrequentSets:list)->list:
        """
        Prunes itemsets from the list of itemsets if any of the sets in infrequentSets are subsets of the itemset

        Parameter
        itemsets (list): A list of itemsets to prune
        infrequentSets (list): A list of subsets to check

        Returns
        list: A pruned list of itemsets
        """
        if len(infrequentSets) == 0:
            return itemsets
        else:
            for i in range(len(itemsets)):
                #print("     " +str(round(i / len(itemsets) * 100, 2)) + "%", end="")
                #print("\r", end="")
                for j in range(len(infrequentSets)):
                    if itemsets[i] == None: 
                        break # set has been pruned so all other comparisons can be skipped
                    elif (infrequentSets[j].issubset(itemsets[i])):
                        itemsets[i] = None
            prunedSets = [itemset for itemset in itemsets if itemset is not None] # filter all nonpruned itemsets
            return prunedSets


In [6]:
def displayAssociationRules(associationRules:list):
    """
    Displays each association rule in a df alongside itemset frequency, 
    lift, confidence and support values.

    Parameters:
    associationRules (list): A list of sorted association rules
    """
    ruleList = []
    for associationRule in associationRules:
        rule = [str(associationRule), len(associationRule.itemset), round(associationRule.lift, 2),       associationRule.confidence, associationRule.support]
        ruleList.append(rule)

    pd.set_option('display.max_rows', 1000)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 150)
    df = pd.DataFrame(ruleList, columns=["Rule", "Length", "Lift", "Conf", "Sup"])
    print(df.to_string(index=False))

## Task 3

In [5]:
startTime = time.time()
path = 'task1.csv'
apriori = Apriori(minsup=0.15, minconf=0.8, minlift=0, path=path)
frequentSets, associationRules = apriori.run()
print("Found {} rules from {} itemsets in {} seconds.".format(len(associationRules), len(frequentSets), round(time.time() - startTime, 2)))

Running Apriori...
 a. Finding association rules from 5 transactions and 6 unique items...
     k = 2    
     k = 3    
     k = 4    
     k = 5    
     k = 6    
 b. Generating association rules from frequent itemsets...
 c. Sorting association rules...
Complete.
Found 97 rules from 35 itemsets in 0.0 seconds.


In [6]:
print(frequentSets)

[{'C'}, {'B'}, {'A'}, {'E'}, {'F'}, {'D'}, {'C', 'B'}, {'C', 'A'}, {'C', 'E'}, {'C', 'F'}, {'C', 'D'}, {'B', 'A'}, {'E', 'B'}, {'B', 'F'}, {'B', 'D'}, {'F', 'A'}, {'D', 'A'}, {'F', 'D'}, {'C', 'B', 'A'}, {'C', 'B', 'E'}, {'C', 'B', 'F'}, {'C', 'B', 'D'}, {'C', 'F', 'A'}, {'C', 'D', 'A'}, {'C', 'F', 'D'}, {'B', 'A', 'F'}, {'B', 'D', 'A'}, {'B', 'D', 'F'}, {'F', 'D', 'A'}, {'C', 'B', 'F', 'A'}, {'C', 'B', 'D', 'A'}, {'C', 'B', 'F', 'D'}, {'C', 'F', 'D', 'A'}, {'B', 'F', 'A', 'D'}, {'C', 'B', 'A', 'F', 'D'}]


In [7]:
displayAssociationRules(associationRules)

                         Rule  Length  Lift  Conf  Sup
{'C', 'F', 'D', 'A'} -> {'B'}       5  1.00   1.0  0.2
{'C', 'F', 'D'} -> {'B', 'A'}       5  1.25   1.0  0.2
{'F', 'D', 'A'} -> {'C', 'B'}       5  1.25   1.0  0.2
{'C', 'B', 'D', 'F'} -> {'A'}       5  1.25   1.0  0.2
{'B', 'D', 'A', 'F'} -> {'C'}       5  1.25   1.0  0.2
{'F'} -> {'C', 'B', 'D', 'A'}       5  1.67   1.0  0.2
{'C', 'F'} -> {'B', 'D', 'A'}       5  1.67   1.0  0.2
{'B', 'F'} -> {'C', 'D', 'A'}       5  1.67   1.0  0.2
{'F', 'A'} -> {'C', 'B', 'D'}       5  1.67   1.0  0.2
{'F', 'D'} -> {'C', 'B', 'A'}       5  1.67   1.0  0.2
{'C', 'B', 'F'} -> {'D', 'A'}       5  1.67   1.0  0.2
{'C', 'F', 'A'} -> {'B', 'D'}       5  1.67   1.0  0.2
{'B', 'A', 'F'} -> {'C', 'D'}       5  1.67   1.0  0.2
{'B', 'D', 'F'} -> {'C', 'A'}       5  1.67   1.0  0.2
{'C', 'B', 'A', 'F'} -> {'D'}       5  1.67   1.0  0.2
     {'C', 'D', 'A'} -> {'B'}       4  1.00   1.0  0.6
     {'C', 'F', 'A'} -> {'B'}       4  1.00   1.0  0.2
     {'C',

## Task 4

### Run Apriori on supermarket.csv 

#### Minsup = 0.10

In [8]:
startTime = time.time()
path = 'supermarket.csv'
apriori = Apriori(minsup=0.10, minconf=0.8, minlift=0, path=path)
frequentSets, associationRules = apriori.run()
print("Found {} rules from {} itemsets in {} seconds.".format(len(associationRules), len(frequentSets), round(time.time() - startTime, 2)))

Running Apriori...
 a. Finding association rules from 4627 transactions and 124 unique items...
     k = 2    
     k = 3    
     k = 4    
     k = 5    
     k = 6    
     k = 7    
     k = 8    
 b. Generating association rules from frequent itemsets...
 c. Sorting association rules...
Complete.
Found 8066 rules from 10282 itemsets in 2800.79 seconds.


In [9]:
print(frequentSets)
displayAssociationRules(associationRules)

[{'CIGS-TOBACCO PKTS'}, {'TOTAL = HIGH'}, {'PUDDINGS-DESERTS'}, {'VEGETABLES'}, {'BREAD AND CAKE'}, {'INSECTICIDES'}, {'PET FOOD'}, {'DEPARTMENT1'}, {'CANNED FRUIT'}, {'CANNED VEGETABLES'}, {'TOTAL = LOW'}, {'DEPARTMENT122'}, {'ELECTRICAL'}, {'TISSUES-PAPER PRD'}, {'SMALL GOODS'}, {'HAIRCARE'}, {'BEEF'}, {'CANNED FISH-MEAT'}, {'CLEANERS-POLISHERS'}, {'POULTRY'}, {'DENTAL NEEDS'}, {'PET FOODS'}, {'CHEESE'}, {'WRAPPING'}, {'COOKING OILS'}, {'JAMS-SPREADS'}, {'SOFT DRINKS'}, {'STATIONARY'}, {'TEA'}, {'MARGARINE'}, {'COLD-MEATS'}, {'COFFEE'}, {'SMALL GOODS2'}, {'FROZEN FOODS'}, {'DAIRY FOODS'}, {'PREPARED MEALS'}, {'FRUIT'}, {'MILK-CREAM'}, {'JUICE-SAT-CORD-MS'}, {'BAKE OFF PRODUCTS'}, {'CONFECTIONARY'}, {'LAUNDRY NEEDS'}, {'BREAKFAST FOOD'}, {'DEODORANTS-SOAP'}, {'DEPARTMENT137'}, {'BISCUITS'}, {'SAUCES-GRAVY-PKLE'}, {'PARTY SNACK FOODS'}, {'LAMB'}, {'POTATOES'}, {'BABY NEEDS'}, {'BAKING NEEDS'}, {'CIGS-TOBACCO PKTS', 'BREAD AND CAKE'}, {'TOTAL = HIGH', 'VEGETABLES'}, {'TOTAL = HIGH', 'BR

#### Minsup = 0.20

In [12]:
startTime = time.time()
path = 'supermarket.csv'
apriori = Apriori(minsup=0.20, minconf=0.8, minlift=0, path=path)
frequentSets, associationRules = apriori.run()
print("Found {} rules from {} itemsets in {} seconds.".format(len(associationRules), len(frequentSets), round(time.time() - startTime, 2)))

Running Apriori...
 a. Finding association rules from 4627 transactions and 124 unique items...
     k = 2    
     k = 3    
     k = 4    
     k = 5    
     k = 6    
 b. Generating association rules from frequent itemsets...
 c. Sorting association rules...
Complete.
Found 206 rules from 640 itemsets in 18.57 seconds.


In [13]:
print(frequentSets)
displayAssociationRules(associationRules)

[{'TOTAL = HIGH'}, {'VEGETABLES'}, {'BREAD AND CAKE'}, {'DEPARTMENT1'}, {'CANNED FRUIT'}, {'CANNED VEGETABLES'}, {'TOTAL = LOW'}, {'DEPARTMENT122'}, {'TISSUES-PAPER PRD'}, {'SMALL GOODS'}, {'BEEF'}, {'CANNED FISH-MEAT'}, {'CLEANERS-POLISHERS'}, {'DENTAL NEEDS'}, {'PET FOODS'}, {'CHEESE'}, {'WRAPPING'}, {'JAMS-SPREADS'}, {'SOFT DRINKS'}, {'STATIONARY'}, {'MARGARINE'}, {'COFFEE'}, {'SMALL GOODS2'}, {'FROZEN FOODS'}, {'DAIRY FOODS'}, {'PREPARED MEALS'}, {'FRUIT'}, {'MILK-CREAM'}, {'JUICE-SAT-CORD-MS'}, {'CONFECTIONARY'}, {'LAUNDRY NEEDS'}, {'BREAKFAST FOOD'}, {'DEODORANTS-SOAP'}, {'DEPARTMENT137'}, {'BISCUITS'}, {'SAUCES-GRAVY-PKLE'}, {'PARTY SNACK FOODS'}, {'BAKING NEEDS'}, {'TOTAL = HIGH', 'VEGETABLES'}, {'TOTAL = HIGH', 'BREAD AND CAKE'}, {'TOTAL = HIGH', 'TISSUES-PAPER PRD'}, {'PET FOODS', 'TOTAL = HIGH'}, {'CHEESE', 'TOTAL = HIGH'}, {'MARGARINE', 'TOTAL = HIGH'}, {'FROZEN FOODS', 'TOTAL = HIGH'}, {'TOTAL = HIGH', 'FRUIT'}, {'TOTAL = HIGH', 'MILK-CREAM'}, {'JUICE-SAT-CORD-MS', 'TOTAL 

#### Minsup = 0.25

In [5]:
startTime = time.time()
path = 'supermarket.csv'
apriori = Apriori(minsup=0.25, minconf=0.8, minlift=0, path=path)
frequentSets, associationRules = apriori.run()
print("Found {} rules from {} itemsets in {} seconds.".format(len(associationRules), len(frequentSets), round(time.time() - startTime, 2)))

Running Apriori...
 a. Finding association rules from 4627 transactions and 124 unique items...
     k = 2    
     k = 3    
     k = 4    
     k = 5    
 b. Generating association rules from frequent itemsets...
 c. Sorting association rules...
Complete.
Found 53 rules from 247 itemsets in 4.99 seconds.


In [6]:
print(frequentSets)
displayAssociationRules(associationRules)

[{'VEGETABLES'}, {'WRAPPING'}, {'SAUCES-GRAVY-PKLE'}, {'TISSUES-PAPER PRD'}, {'BREAD AND CAKE'}, {'PARTY SNACK FOODS'}, {'CHEESE'}, {'BISCUITS'}, {'CANNED VEGETABLES'}, {'TOTAL = LOW'}, {'JAMS-SPREADS'}, {'STATIONARY'}, {'BEEF'}, {'CONFECTIONARY'}, {'JUICE-SAT-CORD-MS'}, {'CANNED FRUIT'}, {'FRUIT'}, {'PET FOODS'}, {'FROZEN FOODS'}, {'LAUNDRY NEEDS'}, {'TOTAL = HIGH'}, {'BAKING NEEDS'}, {'PREPARED MEALS'}, {'BREAKFAST FOOD'}, {'MARGARINE'}, {'SOFT DRINKS'}, {'DEPARTMENT137'}, {'MILK-CREAM'}, {'DAIRY FOODS'}, {'CLEANERS-POLISHERS'}, {'SAUCES-GRAVY-PKLE', 'VEGETABLES'}, {'TISSUES-PAPER PRD', 'VEGETABLES'}, {'BREAD AND CAKE', 'VEGETABLES'}, {'VEGETABLES', 'PARTY SNACK FOODS'}, {'CHEESE', 'VEGETABLES'}, {'BISCUITS', 'VEGETABLES'}, {'CANNED VEGETABLES', 'VEGETABLES'}, {'TOTAL = LOW', 'VEGETABLES'}, {'BEEF', 'VEGETABLES'}, {'JUICE-SAT-CORD-MS', 'VEGETABLES'}, {'FRUIT', 'VEGETABLES'}, {'PET FOODS', 'VEGETABLES'}, {'VEGETABLES', 'FROZEN FOODS'}, {'TOTAL = HIGH', 'VEGETABLES'}, {'VEGETABLES', 'B

# Task 5
Extended Apriori with minRelativeSup constraint

In [7]:
import csv
import itertools
import time
import pandas as pd
from AssociationRule import AssociationRule

class ExtendedApriori():
    """
    This object implements the Apriori unsupervised machine learning algorithm 
    for frequent item set mining and association rule learning
    """
    def __init__(self, minsup:float, minconf:float, minlift:float, minRelativeSup:float, path:str):
        """Initalise the Apiori class

        Parameters:
        minsup (float): Minimuim support
        minconf (float): Minimum confidence
        minlift (float): Minimum lift
        minRelativeSup (float): Minimum relative support
        path (str): Transaction CSV path - each line is transaction of items seperated by comma
        items (set): The set of unique items found across all transactions - initalised by generateUniqueItemSet()
        transactions (list): A list of sets for each transaction - where each is a set of items

        Returns:
        None:Returning value
        """
        self.minsup = minsup
        self.minconf = minconf
        self.minlift = minlift
        self.minRelativeSup = minRelativeSup
        self.path = path
        self.transactions = []
        self.items = []
        self.importTransactions()
        self.generateUniqueItemSet()

    def run(self):
        """
        Runs the Apriori itemset frequency algorithm and returns a sorted list of association rules that pass the minsup, minconf and minlift rules
        """
        print("Running Apriori...")
        print(" a. Finding association rules from {} transactions and {} unique items...".format(len(self.transactions), len(self.items)))
        frequentSets = self.generateFrequentSets() # find all frequent itemsets of length 1 to k-1
        print(" b. Generating association rules from frequent itemsets...")
        associationRules = self.generateAssociationRules(frequentSets) # generate rules that pass the minsup, minlift and minsup thresholds
        print(" c. Sorting association rules...")
        associationRules = self.sortAssociationRules(associationRules) # sorts rules by length, lift, confidence and support
        print("Complete.")
        return frequentSets, associationRules

    def generateAssociationRules(self, frequentSets:list)->list:
        """
        Generates a list of association rules that meet the confidence and lift thresholds

        Parameters:
        frequentSets (list): A list of frequent itemsets

        Returns:
        list: A list containing the association rules that pass the support, confidence and lift thresholds
        """
        associationRules = []
        # partition the set  
        for itemset in frequentSets:
            for i in range(len(itemset)):
                for c in [c for c in itertools.combinations(itemset, i+1)]:
                    body = set(list(c))
                    head = set([i for i in itemset if not i in c])
                    if (len(head) > 0 and len(body) > 0):
                        # from head/body partitions create association rules 
                        associationRule = AssociationRule(body, head) # X -> Y
                        associationRule.support = self.calculateSupport(associationRule.itemset)
                        associationRule.confidence = self.calculateConfidence(associationRule.itemset, associationRule.body)
                        associationRule.lift = self.calculateLift(associationRule.body, associationRule.head, associationRule.confidence)
                        # test is rule passes minconf, minsup and minlift
                        if (associationRule.confidence >= self.minconf) and (associationRule.support >= self.minsup) and (associationRule.lift >= self.minlift):
                            associationRules.append(associationRule)
        return associationRules

    def generateFrequentSets(self)->list:
        """
        Implements the Apriori frequent itemset generation algorithm.
        Where itemsets is c_k and frequentSets = l_k 

        Returns:
        list: A list of frequent itemsets of size k-1
        """
        allFrequentSets = []

        # Generate frequent itemsets of length k
        frequentSets, infrequentSets = self.eliminateCandidates(self.items) # c_1
        for fset in frequentSets:
            allFrequentSets.append(fset)
        # Repeat until no new frequent itemsets are identified
        k = 1
        while True:  
            k += 1
            print("     k = " + str(k) + "    " , end="\n")
            # Generate length (k+1) candidate itemsets from length k frequent itemsets
            itemsets = self.generateItemSets(frequentSets, k)

            # Task 5 extention
            maxSubsetSup = self.calculateMaxSubsetSupport(frequentSets)

            # Prune candidate itemsets containing subsets of length k that are infrequent
            itemsets = self.prune(itemsets, infrequentSets)
            # Count the support of each candidate by scanning the DB
            # Eliminate candidates that are infrequent, leaving only those that are frequent
            prevFrequentSets = frequentSets.copy() # holds a copy of frequent sets for k-1
            frequentSets, infrequentSets = self.eliminateCandidates(itemsets, maxSubsetSup)
            # Runs until no frequent itemsets are identified
            if len(frequentSets) == 0:
                frequentSets = prevFrequentSets
                break
            # stores all the frequent itemsets of size k
            for fset in frequentSets:
                allFrequentSets.append(fset)
        return allFrequentSets

    def sortAssociationRules(self, associationRules:list)->list: # TODO: sorted output list
        """
        Sorts the given list of association rules by four different attributes.
        Including by itemset length (decreasing), minlift, minconf and minsup (decreasing)

        Paramaters:
        associationRules (list): A list of AssociationRule objects

        Returns:
        list: A sorted list of AssociationRule objects

        """
        associationRules = sorted(associationRules, key=lambda x: x.support, reverse=True) # support (decreasing)
        associationRules = sorted(associationRules, key=lambda x: x.confidence, reverse=False) # confidence
        associationRules = sorted(associationRules, key=lambda x: x.lift, reverse=False) # lift value
        associationRules = sorted(associationRules, key=lambda x: len(x.itemset), reverse=True) # number of items (decreasing)

        return associationRules
    
    def eliminateCandidates(self, itemsets:list, maxSubsetSup:float=None)->tuple:
        """
        Sorts candiadate itemsets by calculating the support value and comparing to the mininimum support value

        Parameters:
        itemsets (list): A list of sets to sort

        Returns:
        tuple: Returns a tuple of two sets. Containing the eliminated rules and one containing rules that pass
        """
        frequentSets = []
        infrequentSets = []
        for i in range(len(itemsets)):
            sup = self.calculateSupport(itemsets[i])
            if sup >= self.minsup:
                if len(itemsets) <= 2 or maxSubsetSup == None:
                    frequentSets.append(itemsets[i])
                elif sup >= self.minRelativeSup: # k >= 2
                    frequentSets.append(itemsets[i])
                else:
                    infrequentSets.append(itemsets[i])
            else:
                infrequentSets.append(itemsets[i])
        return frequentSets, infrequentSets

    def generateItemSets(self, items:set, k:int)->list:
        """
        Generates combinations of itemsets and returns a list of sets

        Parameters
        items (set): The set of items to generate combinations from
        k (int): The length of the generated sets

        Returns:
        list: List of the generated combinations of length k
        """
        pairs = list(itertools.combinations(items, 2))

        itemsets = []
        for pair in pairs:
            set1 = pair[0]
            set2 = pair[1]
            set3 = set1.union(set2)
            if set3 not in itemsets and len(set3) == k:
                itemsets.append(set3)
        return itemsets

    def generateUniqueItemSet(self):
        """
        Generates a set of items that appear across all transactions. Updates the class attribute self.transactions
        """
        items = set()
        for transaction in self.transactions:
            for item in transaction:
                if item not in self.items:
                    items.add(item)
        for item in items:
            s = set()
            s.add(item)
            self.items.append(s)

    def importTransactions(self):
        """
        Reads in the csv file of transactions. The file is assumed to have no header. Each row is a set of items contained within a transaction
        """
        with open(self.path, 'r', newline='') as f:
            reader = csv.reader(f)
            for row in reader:
                cleanedRow = []
                for item in row:
                    cleanedRow.append(item.strip().upper())
                self.transactions.append(set(cleanedRow))

    def calculateSupport(self, itemset:set)->float:
        """
        Calculates the support value of a set by dividing the number of transactions a set occurs in with the total number of transactions

        Parameters
        itemset (set): A set of items

        Returns:
        float: The support value for that set (rule)

        """
        return self.count(itemset) / len(self.transactions)

    def calculateRelativeSupport(self, itemset:set, maxSubsetSup:float)->float:
        """
        Calculates the relative support of an itemset against the maximum sort from
        the itemsets of the previous iteration k - 1

        Parameters:
        itemset (set): A set of items
        maxSubsetSup (float): The maximum support for k-1 frequent itemsets

        Returns:
        float: The relative support value

        """
        return self.calculateSupport(itemset) / maxSubsetSup

    def calculateMaxSubsetSupport(self, itemsets:list)->float:
        """
        Finds the maximum support value given a list of itemsets

        Paramaters:
        itemsets (list): A list of itemsets

        Returns:
        float: The maximum support value

        """
        maxSubsetSup = None
        for itemset in itemsets:
            sup = self.calculateSupport(itemset)
            if maxSubsetSup == None:
                maxSubsetSup = sup
            elif sup > maxSubsetSup:
                maxSubsetSup = sup
        return maxSubsetSup

    def calculateConfidence(self, itemset:set, body:set)->float:
        """
        Calculates how often items in Y appear in transactions containing X

        Parameters:
        set (set): A set of items

        Returns:
        float: The confidence value for that set (rule)

        """
        return self.calculateSupport(itemset) / self.calculateSupport(body)

    def calculateLift(self, body:set, head:set, confidence:float)->float:
        """
        Calculates the importance of a rule

        Parameters:
        body (set): The set of items in the association rule body
        head (set): The set of items in the head of the association rule

        Returns:
        float: The lift value of the association rule

        """
        bodySupport = self.calculateSupport(body)
        headSupport = self.calculateSupport(head)
        return confidence / ((bodySupport * headSupport) / bodySupport)

    def count(self, s:set)->int:
        """
        Parameters
        set (set): A set of items

        Returns:
        int: Frequency count for how many times s is a subset of a transaction t
        """
        count = 0
        for transaction in self.transactions:
            if set(s).issubset(transaction):
                count += 1
        return count

    def prune(self, itemsets:list, infrequentSets:list)->list:
        """
        Prunes itemsets from the list of itemsets if any of the sets in infrequentSets are subsets of the itemset

        Parameter
        itemsets (list): A list of itemsets to prune
        infrequentSets (list): A list of subsets to check

        Returns
        list: A pruned list of itemsets
        """

        # TODO: Implement minRelativeSup check


        if len(infrequentSets) == 0:
            return itemsets
        else:
            for i in range(len(itemsets)):
                #print("     " +str(round(i / len(itemsets) * 100, 2)) + "%", end="")
                #print("\r", end="")
                for j in range(len(infrequentSets)):
                    if itemsets[i] == None: 
                        break # set has been pruned so all other comparisons can be skipped
                    elif (infrequentSets[j].issubset(itemsets[i])):
                        itemsets[i] = None
            prunedSets = [itemset for itemset in itemsets if itemset is not None] # filter all nonpruned itemsets
            return prunedSets

def displayAssociationRules(associationRules:list):
    """
    Displays each association rule in a df alongside itemset frequency, 
    lift, confidence and support values.

    Parameters:
    associationRules (list): A list of sorted association rules
    """
    pd.set_option('display.max_rows', 1000)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 150)

    ruleList = []
    for associationRule in associationRules:
        rule = [str(associationRule), len(associationRule.itemset), round(associationRule.lift, 2), associationRule.confidence, associationRule.support]
        ruleList.append(rule)
    df = pd.DataFrame(ruleList, columns=["Rule", "Length", "Lift", "Conf", "Sup"])
    print(df.to_string(index=False))

def main():
    startTime = time.time()

    path = 'task1.csv'
    apriori = ExtendedApriori(minsup=0.15, minconf=0.8, minlift=0, minRelativeSup=0.5, path=path)
    frequentSets, associationRules = apriori.run()

    print("\nComputed association rules:\n")
    displayAssociationRules(associationRules)

    print("\nFound {} rules from {} itemsets in {} seconds.\n".format(len(associationRules), len(frequentSets), round(time.time() - startTime, 2)))

if __name__ == "__main__":
    main()

Running Apriori...
 a. Finding association rules from 5 transactions and 6 unique items...
     k = 2    
     k = 3    
     k = 4    
     k = 5    
 b. Generating association rules from frequent itemsets...
 c. Sorting association rules...
Complete.

Computed association rules:

                    Rule  Length  Lift  Conf  Sup
{'C', 'A', 'D'} -> {'B'}       4  1.00   1.0  0.6
{'A', 'D'} -> {'C', 'B'}       4  1.25   1.0  0.6
{'C', 'D'} -> {'A', 'B'}       4  1.25   1.0  0.6
{'A', 'B', 'D'} -> {'C'}       4  1.25   1.0  0.6
{'C', 'B', 'D'} -> {'A'}       4  1.25   1.0  0.6
{'D'} -> {'C', 'A', 'B'}       4  1.67   1.0  0.6
{'C', 'A'} -> {'B', 'D'}       4  1.67   1.0  0.6
{'B', 'D'} -> {'C', 'A'}       4  1.67   1.0  0.6
{'C', 'A', 'B'} -> {'D'}       4  1.67   1.0  0.6
     {'C', 'A'} -> {'B'}       3  1.00   1.0  0.6
     {'A', 'D'} -> {'B'}       3  1.00   1.0  0.6
     {'C', 'D'} -> {'B'}       3  1.00   1.0  0.6
     {'D'} -> {'A', 'B'}       3  1.25   1.0  0.6
     {'B', 'D'} -

In [12]:
startTime = time.time()
path = 'supermarket.csv'
apriori = ExtendedApriori(minsup=0.15, minconf=0.8, minlift=0, minRelativeSup=0.3, path=path)
frequentSets, associationRules = apriori.run()
print("Found {} rules from {} itemsets in {} seconds.".format(len(associationRules), len(frequentSets), round(time.time() - startTime, 2)))
displayAssociationRules(associationRules)

Running Apriori...
 a. Finding association rules from 4627 transactions and 124 unique items...
     k = 2    
     k = 3    
     k = 4    
 b. Generating association rules from frequent itemsets...
 c. Sorting association rules...
Complete.
Found 17 rules from 133 itemsets in 2.73 seconds.
                                                  Rule  Length  Lift     Conf      Sup
{'FROZEN FOODS', 'BAKING NEEDS'} -> {'BREAD AND CAKE'}       3  1.12 0.809264 0.320942
         {'FRUIT', 'VEGETABLES'} -> {'BREAD AND CAKE'}       3  1.13 0.811509 0.387076
  {'BAKING NEEDS', 'VEGETABLES'} -> {'BREAD AND CAKE'}       3  1.13 0.813751 0.342771
    {'VEGETABLES', 'MILK-CREAM'} -> {'BREAD AND CAKE'}       3  1.14 0.818765 0.358332
  {'FROZEN FOODS', 'VEGETABLES'} -> {'BREAD AND CAKE'}       3  1.14 0.822529 0.334558
       {'FRUIT', 'BAKING NEEDS'} -> {'BREAD AND CAKE'}       3  1.14 0.823158 0.338016
    {'BISCUITS', 'BAKING NEEDS'} -> {'BREAD AND CAKE'}       3  1.15 0.825397 0.314675
         {'